In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [6]:
class ObliviousDecisionTreeLayer(nn.Module):
    def __init__(self, input_dim, num_trees=10, tree_depth=6):
        super(ObliviousDecisionTreeLayer, self).__init__()
        self.num_trees = num_trees
        self.tree_depth = tree_depth
        self.weight = nn.Parameter(torch.randn(num_trees, tree_depth, input_dim))
        self.bias = nn.Parameter(torch.randn(num_trees, tree_depth))  # Bias berbentuk [num_trees, tree_depth]

    def forward(self, x):
        batch_size = x.size(0)
        outputs = []

        for i in range(self.num_trees):
            out = x
            for j in range(self.tree_depth):
                # Perluas 'bias' agar sesuai dengan dimensi batch
                bias = self.bias[i, j].unsqueeze(0).expand(batch_size, -1)  # Bias diperluas agar sesuai dengan batch_size
                # Operasikan linear layer dengan dimensi yang cocok
                decision = torch.sigmoid(F.linear(out, self.weight[i, j].unsqueeze(0), bias))
                # Sesuaikan dimensi 'out' untuk melanjutkan operasi
                out = decision * out
            outputs.append(out)

        # Rata-rata output dari semua pohon
        return torch.mean(torch.stack(outputs), dim=0)

# Definisi NODE Model
class NODE(nn.Module):
    def __init__(self, input_dim, output_dim, num_trees=10, tree_depth=6, hidden_dim=64):
        super(NODE, self).__init__()
        self.tree_layer = ObliviousDecisionTreeLayer(input_dim, num_trees=num_trees, tree_depth=tree_depth)
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # Sesuaikan input dimension sesuai dengan output dari tree_layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.tree_layer(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Persiapan data
data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Konversi ke tensor
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# Inisialisasi model NODE
input_dim = X_train.shape[1]
output_dim = len(set(y))
model = NODE(input_dim, output_dim, num_trees=10, tree_depth=6, hidden_dim=64)

# Definisi optimizer dan loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluasi model
model.eval()
with torch.no_grad():
    y_pred_train = torch.argmax(model(X_train_t), dim=1)
    y_pred_test = torch.argmax(model(X_test_t), dim=1)

    train_acc = accuracy_score(y_train, y_pred_train.numpy())
    test_acc = accuracy_score(y_test, y_pred_test.numpy())
    f1 = f1_score(y_test, y_pred_test.numpy(), average='weighted')

    print(f'Train Accuracy: {train_acc:.4f}')
    print(f'Test Accuracy: {test_acc:.4f}')
    print(f'F1 Score: {f1:.4f}')

Epoch [10/100], Loss: 1.0929
Epoch [20/100], Loss: 1.0782
Epoch [30/100], Loss: 0.9923
Epoch [40/100], Loss: 0.6962
Epoch [50/100], Loss: 0.4592
Epoch [60/100], Loss: 0.2658
Epoch [70/100], Loss: 0.1123
Epoch [80/100], Loss: 0.0710
Epoch [90/100], Loss: 0.0583
Epoch [100/100], Loss: 0.0552
Train Accuracy: 0.9905
Test Accuracy: 1.0000
F1 Score: 1.0000


In [ ]:
class NODEModel(nn.Module):
    """
    Neural Oblivious Decision Ensembles (NODE) model.
    """
    def __init__(self, input_dim, num_trees, depth):
        """
        Initialize the NODE model.
        
        Args:
            input_dim (int): Number of input features.
            num_trees (int): Number of decision trees in the ensemble.
            depth (int): Depth of each decision tree.
        """
        super(NODEModel, self).__init__()
        self.num_trees = num_trees
        self.depth = depth
        
        # Create a list of decision trees
        self.trees = nn.ModuleList([nn.Sequential(
            nn.Linear(input_dim, 2 ** depth),
            nn.ReLU(),
            nn.Linear(2 ** depth, 1)
        ) for _ in range(num_trees)])